In [81]:
import rebound as rb
import celmech as cm
# import scipy
import numpy as np
import matplotlib.pyplot as plt
import radvel
import pandas as pd
import reboundx

# make font bigger
plt.rc('font', size=16)

In [82]:
hd_data = pd.read_csv('hd45364_rvs.csv', sep =';')
hd_data.BJD -= 2.4e6

In [83]:
#Least squares fit: 
fit_params = [ 2.28513053e+02, 7.27743508e+00, 5.39371890e+04, -1.78110513e-01, 
              -4.66450116e-02, 3.43378023e+02, 1.78603199e+01, 5.40186750e+04,
               1.32165040e-01, 9.73129308e-02, -5.29039776e-01] 

# star mass, g and auday to m/s conversion factor
STAR_MASS = 920  # 920 jupiter masses
G = 2.825e-7  # converting G to jupiter masses, au, and days
AUDAY_MS = 1.731e6  # conversion factor for au/day to m/s

def mass_to_semiamp(planet_mass, star_mass, period, eccentricity, inclination):
    """
    planet mass (jupiter masses) to semi amplitude
    """
    return ((2 * np.pi * G/period) ** (1/3) * (planet_mass * np.sin(inclination) / star_mass ** (2/3)) * (1/np.sqrt(1 - eccentricity ** 2)))

def semiamp_to_mass(semiamp, star_mass, period, eccentricity, inclination):
    """
    semi amplitude to planet mass (jupiter masses)
    """
    return (((2 * np.pi * G/period) ** (-1/3)) * (semiamp / np.sin(inclination)) * np.sqrt(1 - eccentricity ** 2) * (star_mass ** (2/3)))


def get_sim_from_params(params, inclination, integrator, time_base, star_mass = STAR_MASS, auday_ms = AUDAY_MS):
    """
    takes in params array, returns a rebound Simulation object with those parameters
    
    param params: numpy array of params:
    
    for i in range(0, num_planets):
    
    params[i + 0] is period
    params[i + 1] is semiamp
    params[i + 2] is tc (time of conjunction)
    params[i + 3] is sqrt(e) * cos(omega)
    params[i + 4] is sqrt(e) * sin(omega)
    
    params[5 * num_planets] is rv offset
    params[5 * num_planets + 1] is jit
    
    param inclination: inclination of system in the observation plane (pi/2 is in the plane of the sky, 0 is edge-on)
    param integrator: integrator to use, one of 'whfast' or 'ias15'
    param time_base: base time (to begin integration from) in the simulation
    """
    
    num_planets = int((len(params) - 1) / 5) # -2 because there are rv_offset and jit parameters:
    
    sim = rb.Simulation()
    sim.integrator = integrator
    sim.t = time_base  # keplerian and n-body models initialized at the same time offset
    # print(sim.t)
    if integrator == 'whfast':  # if using whfast integrator, set timestep
        sim.dt = 1/50 * min(params[0::5][:-1])  # timestep is 1/20th of the shortest orbital period of any planet
        # print(sim.dt)
    sim.units = ('AU', 'Mjupiter', 'day')

    sim.add(m = star_mass)  # star mass as a constant
        
    for i in range (0, num_planets):
        # print(i)
        # planet parameters
        period = params[5*i]  # in days
        semiamp = params[5*i + 1] / auday_ms # divide by auday_ms because semiamp given in m/s
        eccentricity = params[5*i + 3] ** 2 + params[5*i + 4] ** 2  # eccentricity from secos, sesin
        omega = np.arctan2(params[5*i + 3], params[5*i + 4])  # omega from arctan of sesin, secos
        # get tp by converting from tc
        tp = radvel.orbit.timetrans_to_timeperi(params[5*i + 2], per = period, ecc = eccentricity, omega = omega)
        
        # mass
        mass = semiamp_to_mass(semiamp = semiamp, star_mass = star_mass, period = period, eccentricity = eccentricity, inclination = inclination)
        
        # adding to simulation
        sim.add(m = mass, P = period, e = eccentricity, T = tp, omega = omega, inc = inclination)
        
    sim.move_to_com()  # move to center of mass
    
    return sim

def get_simple_sim(masses, integrator = 'ias15', period_ratio = 3/2, epsilon=0.01):
    """
    gets simple sim (for eccentricity track stuff)
    param masses: array of planet masses
    param integrator: integrator
    param epsilon: amount by which the resonant period ratio should be offset from the equilibrium in the simulation
    """
    sim = rb.Simulation()
    sim.integrator = integrator
    # central star
    sim.add(m = 1)
    
    sim.add(m = masses[0], P = 1)
    sim.add(m = masses[1], P = period_ratio * (1 + epsilon))

    sim.move_to_com()
    if integrator == 'whfast':
        sim.dt = 1/50 * 1  # dy default use 1/50th of the inner planet's orbital period for the timestep if using whfast
    return sim

In [84]:
gsim = get_sim_from_params(fit_params, inclination = np.pi/2, integrator= 'ias15', time_base = 0)

masses = [gsim.particles[1].m, gsim.particles[2].m]
sim = get_simple_sim(masses, integrator = 'ias15')

# create pvars, pham just using the simple sim with interaction terms
pvars = cm.Poincare.from_Simulation(sim)
pham = cm.PoincareHamiltonian(pvars)
# add interaction terms
pham.add_MMR_terms(3, 1, max_order = 1, inclinations=False)


In [85]:
import sympy as sp
# defining stuff:

# action variables
# Lambda1, Lambda2, Gamma1, Gamma2, Q1, Q2 = sp.symbols('Lambda_1, Lambda_2, Gamma_1, Gamma_2, Q_1, Q_2')
# old angle coordinates
lambda1, lambda2, pomega1, pomega2, Omega1, Omega2 = sp.symbols('lambda_1, lambda_2, varpi_1, varpi_2, Omega_1, Omega_2')
# new angle coordinates
theta1, theta2, l, psi, phi1, phi2 = sp.symbols('theta_1, theta_2, l, psi, phi_1, phi_2')
p1, p2, L, Psi, Phi1, Phi2 = sp.symbols('p_1, p_2, L, Psi, Phi_1, Phi_2')

Canonical transformation stuff:

In [86]:
# converts
# lambda_1, lambda_2, pomega_1, pomega_2, Omega_1, Omega_2 to
# theta_1, theta_2, l, psi, phi_1, phi_2
pmatrix = [[-2, 3, 1, 0, 0, 0], [-2, 3, 0, 1, 0, 0], [-2, 3, 0, 0, 0, 0], 
           [1, -1, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1]]


# eta, kappa -> p, theta canonical transformation
ctptheta = cm.CanonicalTransformation.from_poincare_angles_matrix(pvars, pmatrix, 
                                                            new_qp_pairs = [(theta1, p1), (theta2, p2), 
                                                                            (l, L), (psi, Psi), 
                                                                            (phi1, Phi1), (phi2, Phi2)])
# p theta hamiltonian
ham_ptheta = ctptheta.old_to_new_hamiltonian(pham, do_reduction=True)

# p, theta -> x, y canonical transformation
ctxy = cm.CanonicalTransformation.polar_to_cartesian(ham_ptheta.full_qp_vars,indices=[0,1])
# x, y hamiltonian
ham_xy = ctxy.old_to_new_hamiltonian(ham_ptheta)#, do_reduction=True)

# composite transformation that goes from eta, kappa -> x, y
ctcomp = cm.CanonicalTransformation.composite([ctptheta, ctxy])

In [87]:
ham_ptheta.H, ham_xy.H

⎛                                                                    ____     
⎜  √2⋅C_{(3,-2,-1,0,0,0)}_{1,2})__{(0,0,0,0),(0,0)}(\alpha⋅G⋅m₁⋅m₂⋅╲╱ p₁ ⋅cos(
⎜- ───────────────────────────────────────────────────────────────────────────
⎜                              _______________                                
⎝                            ╲╱ \Lambda_{1,0} ⋅a_{2,0}                        

                                                                        ____  
θ₁)   √2⋅C_{(3,-2,0,-1,0,0)}_{1,2})__{(0,0,0,0),(0,0)}(\alpha⋅G⋅m₁⋅m₂⋅╲╱ p₂ ⋅c
─── - ────────────────────────────────────────────────────────────────────────
                                  _______________                             
                                ╲╱ \Lambda_{2,0} ⋅a_{2,0}                     

                   2   2   3                    2   2   3                     
os(θ₂)            G ⋅M₁ ⋅μ₁                    G ⋅M₂ ⋅μ₂             C_{(3,-2,
────── - ─────────────────────────── - ───────────

Conserved quantities:

In [88]:
L, Psi, ctcomp.new_to_old(L), ctcomp.new_to_old(Psi)

⎛                  2     2     2     2             ⎞
⎜                η₁    η₂    κ₁    κ₂              ⎟
⎜L, Ψ, Λ₁ + Λ₂ - ─── - ─── - ─── - ───, 3⋅Λ₁ + 2⋅Λ₂⎟
⎝                 2     2     2     2              ⎠

### Dissipative:

Add dissipative forces:

In [89]:
# symbols
tau_a_1, tau_e_1 = sp.symbols('tau_a_1, tau_e_1')
tau_a_2, tau_e_2 = sp.symbols('tau_a_2, tau_e_2')
Lambda1 = pham.qp_vars[6]
Lambda2 = pham.qp_vars[9]
eta_1 = pham.qp_vars[1]
eta_2 = pham.qp_vars[4]
kappa_1 = pham.qp_vars[7]
kappa_2 = pham.qp_vars[10]
t = sp.symbols('t')

# dLambda, deta, dkappa equations:

dLambda_1 = -Lambda1/(2 * tau_a_1)
dLambda_2 = -Lambda2/(2 * tau_a_2)

deta_1 = -eta_1 * (1/(4 * tau_a_1) + 1/tau_e_1)
deta_2 = -eta_2 * (1/(4 * tau_a_2) + 1/tau_e_2)

dkappa_1 = -kappa_1 * (1/(4 * tau_a_1) + 1/tau_e_1)
dkappa_2 = -kappa_2 * (1/(4 * tau_a_2) + 1/tau_e_2)

dLambda_1, deta_1, dkappa_1

⎛ -Λ₁        ⎛ 1       1   ⎞      ⎛ 1       1   ⎞⎞
⎜──────, -η₁⋅⎜──── + ──────⎟, -κ₁⋅⎜──── + ──────⎟⎟
⎝2⋅τₐ ₁      ⎝τₑ ₁   4⋅τₐ ₁⎠      ⎝τₑ ₁   4⋅τₐ ₁⎠⎠

In [90]:
dissipative = sp.zeros(len(pham.values), 1)

# add lambda, eta, kappa dissipative terms
dissipative[6] = dLambda_1
dissipative[9] = dLambda_2

dissipative[1] = deta_1
dissipative[4] = deta_2

dissipative[7] = dkappa_1
dissipative[10] = dkappa_2

dissipative

⎡         0         ⎤
⎢                   ⎥
⎢    ⎛ 1       1   ⎞⎥
⎢-η₁⋅⎜──── + ──────⎟⎥
⎢    ⎝τₑ ₁   4⋅τₐ ₁⎠⎥
⎢                   ⎥
⎢         0         ⎥
⎢                   ⎥
⎢         0         ⎥
⎢                   ⎥
⎢    ⎛ 1       1   ⎞⎥
⎢-η₂⋅⎜──── + ──────⎟⎥
⎢    ⎝τₑ ₂   4⋅τₐ ₂⎠⎥
⎢                   ⎥
⎢         0         ⎥
⎢                   ⎥
⎢       -Λ₁         ⎥
⎢      ──────       ⎥
⎢      2⋅τₐ ₁       ⎥
⎢                   ⎥
⎢    ⎛ 1       1   ⎞⎥
⎢-κ₁⋅⎜──── + ──────⎟⎥
⎢    ⎝τₑ ₁   4⋅τₐ ₁⎠⎥
⎢                   ⎥
⎢         0         ⎥
⎢                   ⎥
⎢       -Λ₂         ⎥
⎢      ──────       ⎥
⎢      2⋅τₐ ₂       ⎥
⎢                   ⎥
⎢    ⎛ 1       1   ⎞⎥
⎢-κ₂⋅⎜──── + ──────⎟⎥
⎢    ⎝τₑ ₂   4⋅τₐ ₂⎠⎥
⎢                   ⎥
⎣         0         ⎦

Define $(D, L_2)$ new variables s.t.
$$
\begin{pmatrix}
L\\
\Psi
\end{pmatrix}
= 
\begin{pmatrix}
-1 & 1+\rho\\
0 & j\rho+(j-1)
\end{pmatrix}
\cdot
\begin{pmatrix}
D\\
L_2
\end{pmatrix}
$$
where $\rho = \left(\frac{\Lambda_1}{\Lambda_2}\right)_\mathrm{res} \approx \frac{m_1}{m_2}\left(\frac{j-1}{j}\right)^{1/3}$.

In terms of old variables,
$$
L = \Lambda_{1} + \Lambda_{2} - \frac{\eta_{1}^{2}}{2} - \frac{\eta_{2}^{2}}{2} - \frac{\kappa_{1}^{2}}{2} - \frac{\kappa_{2}^{2}}{2} = (1 + \rho)L_2 - D\\
\Psi = j \Lambda_{1} + (j-1) \Lambda_{2} = (j\rho+(j-1))L_2~.
$$
In other words, $L_2$ represents the value of $\Lambda_2$ when the planets' instantaneous period ratio is at the exact resonant value and $D$ is the value of $(p_1+p_2)$ when $\Lambda_2 = L_2$.

The planets' period ratio can be expressed in terms of 
$$
\Delta = \frac{j-1}{j}\frac{P_2}{P_1} - 1 \approx 3\left(\frac{\Lambda_2 - L_2}{L_2} - \frac{\Lambda_1 - \rho L_2}{\rho L_2}\right) = 3\frac{\delta\Lambda_2}{L_2}\left(1+\frac{j-1}{j\rho}\right)
$$
where $\delta\Lambda_2 = \Lambda_2 - L_2$

Therefore, we can write 
$$
D = p_1+p_2+(1+\rho)L_2-\Lambda_1 - \Lambda_2
$$
Or
$$
\begin{eqnarray}
D &=& p_1+p_2 - \frac{1}{j}\delta\Lambda_2\\
  &=& p_1+p_2 - \frac{1}{3}\left(\frac{\rho L_2}{j\rho+j-1}\right)\Delta
\end{eqnarray}
$$

The dissipative evolution of $D$ is therefore
$$
\begin{eqnarray}
\dot{D}\big|_\mathrm{dis} &=&-\frac{p_1}{\tau_{e1}}-\frac{p_2}{\tau_{e2}}- \frac{1}{3}\left(\frac{\rho L_2}{j\rho+j-1}\right)\dot{\Delta}\big|_\mathrm{dis}\\
&=&
-\frac{p_1}{\tau_{e1}}-\frac{p_2}{\tau_{e2}}+\frac{1}{2}\left(\frac{\rho L_2}{j\rho+j-1}\right)\frac{1}{\tau_\alpha}
\end{eqnarray}
$$
where $\tau_\alpha^{-1} = \tau_{a,1}^{-1}- \tau_{a,2}^{-1}$

Introduce $D$, $L_2$, into the dissipative equation, using $\rho$ and the matrix equation above. Since this is a $3:2$ resonance, $j = 3$:

In [91]:
D, L2, rho = sp.symbols("D, L2, rho")
# Matrix expression for D and L2:
j = 3
transformation_matrix = sp.Matrix([[-1, 1 + rho], [0, j * rho + j - 1]])
# Solve matrix to get values of D, L2:
d_l2_matrix = transformation_matrix.LUsolve(sp.Matrix([L, Psi]))

Transform to new variables:

In [92]:
ctcomp.old_qp_vars

[λ₁, η₁, ρ₁, λ₂, η₂, ρ₂, Λ₁, κ₁, σ₁, Λ₂, κ₂, σ₂]

In [93]:
_, _, _, _, _, _, Lambda1, _, _, Lambda2, _, _, = ctcomp.old_qp_vars
y1, y2, _, _, _, _, x1, x2, L, Psi, _, _ = ctcomp.new_qp_vars

# y1, y2, x1, x2, D
newvars = [y1, y2, x1, x2, d_l2_matrix[0]]
newvars

⎡                     Ψ⋅(ρ + 1)⎤
⎢y₁, y₂, x₁, x₂, -L + ─────────⎥
⎣                      3⋅ρ + 2 ⎦

Use Jacobian:

In [94]:
ct_jac = sp.simplify(sp.Matrix([[sp.diff(ctcomp.new_to_old(newvar), oldvar) for oldvar in pham.qp_vars] for newvar in newvars]))

In [95]:
d_l2_dissipative = sp.simplify(ctcomp.old_to_new(ct_jac * dissipative))
d_l2_dissipative

⎡                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                   

Now substitute values of $L$, $\Psi$ into the dissipative matrix:

In [96]:
L_Psi_matrix = transformation_matrix * sp.Matrix([D, L2])  # values of L and Psi in terms of D and L2
# subsitute
d_l2_dissipative_N = sp.simplify(d_l2_dissipative.subs([(L, L_Psi_matrix[0]), (Psi, L_Psi_matrix[1])]))
d_l2_dissipative_N

⎡                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                   

Now approximate $\dot{D}$ as $\dot{D}\big|_\mathrm{dis} = -\frac{p_1}{\tau_{e1}}-\frac{p_2}{\tau_{e2}}+\frac{1}{2}\left(\frac{\rho L_2}{j\rho+j-1}\right)\frac{1}{\tau_\alpha}$, which gives us:

In [97]:
# definition of tau_a
tau_a = 1/(1/tau_a_1 - 1/tau_a_2)
# compute d dot
Ddot_ptheta = -p1/tau_e_1 - p2/tau_e_2 + 1/2 * (rho * L2)/(j * rho + j - 1) * 1/tau_a
Ddot_ptheta

         ⎛   1      1  ⎞              
0.5⋅L₂⋅ρ⋅⎜- ──── + ────⎟              
         ⎝  τₐ ₂   τₐ ₁⎠    p₁     p₂ 
──────────────────────── - ──── - ────
        3⋅ρ + 2            τₑ ₁   τₑ ₂

And transform from $(p, \theta)$ to $(x, y)$ coordinates:

In [98]:
Ddot_xy = ctxy.old_to_new(Ddot_ptheta)
Ddot_xy

                            2     2     2     2
0.5⋅L₂⋅ρ⋅(-τₐ ₁ + τₐ ₂)   x₂  + y₂    x₁  + y₁ 
─────────────────────── - ───────── - ─────────
  τₐ ₁⋅τₐ ₂⋅(3⋅ρ + 2)       2⋅τₑ ₂      2⋅τₑ ₁ 

Use the $\dot{D}$ expression in the dissipative part to get our expression:

In [99]:
# set Ddot as the approximate expression for Ddot and keep everything else the same
d_l2_dissipative_Ddot_approx = sp.Matrix([Ddot_xy if i == 4 else d_l2_dissipative_N[i] for i in range(0, len(d_l2_dissipative_N))])
d_l2_dissipative_Ddot_approx

⎡                   y₁      y₁                  ⎤
⎢                - ──── - ──────                ⎥
⎢                  τₑ ₁   4⋅τₐ ₁                ⎥
⎢                                               ⎥
⎢                   y₂      y₂                  ⎥
⎢                - ──── - ──────                ⎥
⎢                  τₑ ₂   4⋅τₐ ₂                ⎥
⎢                                               ⎥
⎢                   x₁      x₁                  ⎥
⎢                - ──── - ──────                ⎥
⎢                  τₑ ₁   4⋅τₐ ₁                ⎥
⎢                                               ⎥
⎢                   x₂      x₂                  ⎥
⎢                - ──── - ──────                ⎥
⎢                  τₑ ₂   4⋅τₐ ₂                ⎥
⎢                                               ⎥
⎢                            2     2     2     2⎥
⎢0.5⋅L₂⋅ρ⋅(-τₐ ₁ + τₐ ₂)   x₂  + y₂    x₁  + y₁ ⎥
⎢─────────────────────── - ───────── - ─────────⎥
⎣  τₐ ₁⋅τₐ ₂⋅(3⋅ρ + 2)       2⋅τₑ ₂      2⋅τₑ ₁ ⎦

### Conservative:

Flow of $xy$-Hamiltonian is the conservative part of dynamics:

In [100]:
ham_xy.flow

⎡                                                                             
⎢  C_{(3,-2,-1,0,0,0)}_{1,2})__{(0,0,0,0),(0,0)}(\alpha⋅G⋅m₁⋅m₂               
⎢- ──────────────────────────────────────────────────────────── - ────────────
⎢                     _______________                                         
⎢                   ╲╱ \Lambda_{1,0} ⋅a_{2,0}                     ⎛   2       
⎢                                                                 ⎝8⋅L  - 8⋅L⋅
⎢                                                                             
⎢                                                                             
⎢  C_{(3,-2,0,-1,0,0)}_{1,2})__{(0,0,0,0),(0,0)}(\alpha⋅G⋅m₁⋅m₂               
⎢- ──────────────────────────────────────────────────────────── - ────────────
⎢                     _______________                                         
⎢                   ╲╱ \Lambda_{2,0} ⋅a_{2,0}                     ⎛   2       
⎢                                                   

Substitute values of $\tau_{a_1}$, $\tau_{a_2}$, $\rho$ and $L_2$ into the flow:

In [101]:
# rho_val defined as ratio of resonant lambda2 to lambda1
rho_val = pham.qp[Lambda1]/pham.qp[Lambda2]
# L2 found using the matrix
L2_val = d_l2_matrix[1].subs([(Psi, ham_xy.full_qp[Psi]), (rho, rho_val)])

rho_val, L2_val

(0.18837122893574784, 0.427371058047602)

In [102]:
cons_flow_xy = ham_xy.flow.subs([(L, L_Psi_matrix[0]), (Psi, L_Psi_matrix[1]), (rho, rho_val), (L2, L2_val)]).subs(ham_xy.H_params)
cons_flow_xy

⎡                                                                             
⎢                                                                             
⎢- ───────────────────────────────────────────────────────────────────────────
⎢                                                                             
⎢  ⎛                             4         2   2         2   2         2   2  
⎢  ⎝0.730836900228798⋅D + 0.25⋅x₁  + 0.5⋅x₁ ⋅x₂  + 0.5⋅x₁ ⋅y₁  + 0.5⋅x₁ ⋅y₂  +
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢- ───────────────────────────────────────────────────────────────────────────
⎢                                                                             
⎢  ⎛                             4         2   2         2   2         2   2  
⎢  ⎝0.730836900228798⋅D + 0.25⋅x₁  + 0.5⋅x₁ ⋅x₂  + 0

Finally substitute $\rho$ and $L_2$ into the dissipative flow component to get the final dissipative flow:

In [103]:
d_l2_dissipative_Ddot_approx_N = d_l2_dissipative_Ddot_approx.subs([(rho, rho_val), (L2, L2_val)])
d_l2_dissipative_Ddot_approx_N

⎡                         y₁      y₁                        ⎤
⎢                      - ──── - ──────                      ⎥
⎢                        τₑ ₁   4⋅τₐ ₁                      ⎥
⎢                                                           ⎥
⎢                         y₂      y₂                        ⎥
⎢                      - ──── - ──────                      ⎥
⎢                        τₑ ₂   4⋅τₐ ₂                      ⎥
⎢                                                           ⎥
⎢                         x₁      x₁                        ⎥
⎢                      - ──── - ──────                      ⎥
⎢                        τₑ ₁   4⋅τₐ ₁                      ⎥
⎢                                                           ⎥
⎢                         x₂      x₂                        ⎥
⎢                      - ──── - ──────                      ⎥
⎢                        τₑ ₂   4⋅τₐ ₂                      ⎥
⎢                                                           ⎥
⎢    2  

Add a $\dot{D} = 0$ line to the conservative flow to get the final conservative flow:

In [119]:
cons_flow_xy_N = cons_flow_xy.row_insert(len(cons_flow_xy), sp.Matrix([0]))
cons_flow_xy_N

⎡                                                                             
⎢                                                                             
⎢- ───────────────────────────────────────────────────────────────────────────
⎢                                                                             
⎢  ⎛                             4         2   2         2   2         2   2  
⎢  ⎝0.730836900228798⋅D + 0.25⋅x₁  + 0.5⋅x₁ ⋅x₂  + 0.5⋅x₁ ⋅y₁  + 0.5⋅x₁ ⋅y₂  +
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢- ───────────────────────────────────────────────────────────────────────────
⎢                                                                             
⎢  ⎛                             4         2   2         2   2         2   2  
⎢  ⎝0.730836900228798⋅D + 0.25⋅x₁  + 0.5⋅x₁ ⋅x₂  + 0

Final flow (sum of conservative and dissipative) is:

In [124]:
ham_flow = d_l2_dissipative_Ddot_approx_N + cons_flow_xy_N
ham_flow

⎡                                                                             
⎢                                                                             
⎢- ───────────────────────────────────────────────────────────────────────────
⎢                                                                             
⎢  ⎛                             4         2   2         2   2         2   2  
⎢  ⎝0.730836900228798⋅D + 0.25⋅x₁  + 0.5⋅x₁ ⋅x₂  + 0.5⋅x₁ ⋅y₁  + 0.5⋅x₁ ⋅y₂  +
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢- ───────────────────────────────────────────────────────────────────────────
⎢                                                                             
⎢  ⎛                             4         2   2         2   2         2   2  
⎢  ⎝0.730836900228798⋅D + 0.25⋅x₁  + 0.5⋅x₁ ⋅x₂  + 0

Variables are $\mathbf{z} = \left(y_1, y_2, x_1, x_2, D\right)$ where $D$ is conserved. Compute $D$ using the matrix equation above and substitute it into the final sum of conservative and dissipative flows, and substitute $\tau_a$, $\tau_e$ values as well (same values that we used to comapre to the `REBOUND` simulation):